# City of Tacoma Decision Support Example 

This notebook demonstrates how to implement the Promethee II methodology for multicriteria decision support analysis. It combines Equity Index values and pollutant loading values. 

It uses the 'pymcdm' library to implement the decision support methodology. 

## Setup

In [ ]:
#Install pymcdm if not already installed
#!pip install pymcdm

In [ ]:
# Load Required Libraries
import numpy as np
import urllib
import pandas as pd
import geopandas as gp
from pymcdm.methods import promethee

## Load Data

In [ ]:
#set the crs to psuedo mercator for accurate area calculations
crs = {'init': 'epsg:3857'}

The Tacoma Equity Index is available on the REST API. For more information on the Equity Index, see: https://www.cityoftacoma.org/cms/One.aspx?portalId=169&pageId=175030

In [ ]:
#download the Equity Index geojson from the argis rest api 
url = 'https://gis.cityoftacoma.org/arcgis/rest/services/General/Equity2020/MapServer/1/query?where=1%3D1&outFields=*&outSR=3857&f=geojson&returnGeometry=true'
df = gp.read_file(url,driver="GeoJSON")
df.head() 


,OBJECTID,GEOID,PerOpSps,OpSpcZSc,VotP_Ind,FoodScr,FodScZSc,TranRatSc,TranScZSc,VtPaZSc,...,Livability_Text,Economic_Value,Economic_Text,Education_Value,Education_Text,Environmental_Value,Environmental_Text,Opportunity_Value,Opportunity_Text,geometry
0,1,530530634006,1.000000,0.035807,54.734411,0.026282,-0.546987,3.557119,-0.452415,-0.267765,...,Very Low,-0.039804,Moderate,-0.131395,Moderate,-0.011004,Moderate,-0.154915,Moderate,"POLYGON ((-13629591.385 5970472.516, -13629591..."
1,2,530530607003,0.128695,-2.322528,63.015464,0.165382,-0.334391,0.886470,-0.710159,0.497892,...,Moderate,-0.141087,Moderate,0.893900,Very High,0.176006,High,0.124198,High,"POLYGON ((-13633830.569 5983840.367, -13633764..."
2,3,530530616021,0.983953,-0.007627,51.052632,0.921449,0.821155,82.855427,3.000000,-0.608178,...,Very Low,-0.654059,Very Low,0.106653,High,-1.336531,Very Low,-0.583043,Very Low,"POLYGON ((-13629530.371 5982863.673, -13629522..."
3,4,530530617001,1.276362,0.783828,47.265222,0.260630,-0.188817,22.815634,1.406225,-0.958358,...,Low,-0.338728,Low,0.160697,High,-0.583770,Very Low,-0.226218,Low,"POLYGON ((-13630259.381 5980394.052, -13630233..."
4,5,530530617005,1.105945,0.322564,53.955696,0.298483,-0.130964,12.323527,0.393632,-0.339764,...,Low,0.028287,Moderate,-0.271746,Moderate,-0.768931,Very Low,-0.258241,Low,"POLYGON ((-13632298.539 5981774.455, -13632439..."


The equity index has 29 metrics that feed into 5 major categories. We will use the major categories for the decision support module.  

In [ ]:
#select columns pertinent to the major categories 
equity_categories = ['Access',
        'Economic_Value',
        'Environmental_Value', 
        'Livability_Value',
        'Opportunity_Value']
        
equity_index = df.loc[:,equity_categories+['geometry']]     

equity_index.head()



,Access,Economic_Value,Environmental_Value,Livability_Value,Opportunity_Value,geometry
0,0.103123,-0.039804,-0.011004,-0.695493,-0.154915,"POLYGON ((-13629591.385 5970472.516, -13629591..."
1,-0.382689,-0.141087,0.176006,0.074860,0.124198,"POLYGON ((-13633830.569 5983840.367, -13633764..."
2,0.363585,-0.654059,-1.336531,-1.394862,-0.583043,"POLYGON ((-13629530.371 5982863.673, -13629522..."
3,-0.042237,-0.338728,-0.583770,-0.327051,-0.226218,"POLYGON ((-13630259.381 5980394.052, -13630233..."
4,-0.016136,0.028287,-0.768931,-0.262677,-0.258241,"POLYGON ((-13632298.539 5981774.455, -13632439..."


## Spatial processing 

This section calculates area weighted values and summarizes them by subbasin.


### Calculate area of equity index polygons 

In [ ]:
#Calculate the area of each polygon in square meters
equity_index['equity_index_area']=equity_index.area

### Combine with subbasin data

In [ ]:
#get subbsasins 
url = 'https://gis.cityoftacoma.org/arcgis/rest/services/ES/SurfacewaterNetwork/MapServer/41/query?where=1%3D1&outFields=*&outSR=3857&f=geojson'
subbasins = gp.read_file(url)

#add a couple of hypothetical pollutant columns using random numbers
subbasins['TSS_concentration'] = np.random.rand(subbasins.shape[0])
subbasins['TN_concentration'] = np.random.rand(subbasins.shape[0])



### Overlay and summarize

In [ ]:
shp_tmp = subbasins.overlay(equity_index, how='intersection')

#calculate area 
shp_tmp['intersected_area'] = shp_tmp.area
shp_tmp['ratio'] = shp_tmp['intersected_area'] / shp_tmp['equity_index_area']

shp_tmp[equity_categories]=shp_tmp[equity_categories].multiply(shp_tmp["ratio"], axis="index")

# sum over census blocks
df_weighted_avg = shp_tmp.groupby(['SUBBASIN'])[equity_categories].sum() 


Join back with watersheds 


In [ ]:
preference_table = subbasins.merge(df_weighted_avg, on='SUBBASIN')

## Promethee inputs 

### Preference matrix

The preference matrix consists of numerical values of each criterion for each subbasin. Subbasins are rows and criteria  are columns

In [ ]:
criteria_cols = ['Access','Economic_Value','Environmental_Value', 'Livability_Value','Opportunity_Value','TSS_concentration','TN_concentration']

In [ ]:
pref_matrix = preference_table[criteria_cols]
subbasins_df = pd.DataFrame(preference_table['SUBBASIN'].drop(columns='geometry'))

subbasins_df.head()

,SUBBASIN
0,WS_03
1,WS_01
2,FL_03
3,TF_01
4,NE_03


### Weights

Weights reflect the relative importance of the different criteria. The higher the weight, the more important the criteria. Weights can be any non-negative number. 

In [ ]:
# Get a weight for each criterion between 0 and 5
n = pref_matrix.shape[1] # number of criteria
weights = np.random.randint(0,5,n) # random weights

## Types 

Types refer to whether the criteria should be minimized or maximized. 
Equity values should be minimized (priority given to areas with a lower equity index value)

Pollutant values can be either minimized or maximized depending on the type of action being implmented: 

* Retrofit projects should maximize pollutant values (subbasins with high loading would be prioritized)
* Restoration projects should minimze pollutant values (subbasins with low loading would be prioritiezed)

In [ ]:
#User selects a scenario - either retrofit or restoration
retrofit = 1 # 1 for retrofit, 0 for restoration. User selected 

#number of pollutants
n_pollutants = 2

if retrofit == 1:
    #retrofit
    pollutant_types = np.ones(n_pollutants)
else:
    #restoration
    pollutant_types = np.ones(n_pollutants)*-1



Equity criteria should always be minimized indicated that areas with lower equity index values will be prioritized

In [ ]:
# Set the preference direction to negative for all equity criteria
equity_types = np.ones(len(equity_categories))*-1
# join the two types for input into the promethee function
types = np.concatenate((equity_types,pollutant_types))

In [ ]:
from pymcdm.methods import PROMETHEE_II
# promethee has the ability to select different preference functions. For simplicity, we will use the usual linear preference function,
#  meaning that there are no thresholds for indifference or preference.

p_function = 'usual'
body = PROMETHEE_II(p_function)
matrix =  pref_matrix.to_numpy()

scores = [round(preference, 2) for preference in body(matrix, weights, types)]
scores = np.asarray(scores).reshape(-1, 1)  # convert to numpy array


In [ ]:
# normalize scores to 0-100
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
scaled_scores = mms.fit_transform(scores)*100





In [ ]:
score_df = pd.DataFrame({'SUBBASIN':subbasins_df['SUBBASIN'],'score':scaled_scores[:,0]}).sort_values(by='score',ascending=False)
score_df.head()

,SUBBASIN,score
51,FL_09,100.000000
44,FS_08,96.959013
54,FS_09,93.741736
66,FL_10,91.449978
29,FS_04,89.202292


## Bonus - Show results with DeckGL 

In [ ]:
#merge the scores with the subbasin polygons
subbasins_results = subbasins.merge(score_df, on='SUBBASIN').to_crs("EPSG:4326")

In [ ]:
import pydeck 
import geopandas as gpd

INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=47.2529,
  longitude=-122.4443,
  zoom=10,
  max_zoom=16,
  pitch=60,
  bearing=45
)

geojson = pydeck.Layer(
    'GeoJsonLayer',
    data=subbasins_results,
    opacity=1,
    stroked=True,
    filled=True,
    extruded=True,
    wireframe=False,
    get_elevation='score*2.5',
    elevation_scale=10,
    get_fill_color='[200, score*2.5, 200]',
    lineWidthScale = 200,
    lineWidthMinPixels=  6,
    get_line_color=[0,0,0],
    pickable=True
)

r = pydeck.Deck(
    layers=[geojson],
    initial_view_state=INITIAL_VIEW_STATE)

r.to_html()


